In [1]:
from dotenv import load_dotenv
import time
from datetime import datetime
import statistics


load_dotenv()

True

In [2]:
from utils import *

gen = generate_mdpp_data()

In [3]:
total_num_passed = 0
total_num_test = 0
total_num_error = 0

case_err = list()

In [ ]:
start_time = time.time()
execution_times = []
total_num_passed = 0
total_num_test = 0
total_num_error = 0
case_err = []

In [ ]:
print(f"Benchmark started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

for test_set in gen:
    task, code_ground_truth, test_list = test_set
    
    # 각 테스트 케이스의 시작 시간 기록
    case_start_time = time.time()

    function_name = extract_first_function_name(code_ground_truth)
    prompt_with_name = f"다음 문제를 푸는 파이썬 함수를 작성하세요. test_case를 통과하도록 함수를 작성하세요. 함수 이름은 반드시 `{function_name}` 이어야 합니다.\n\n task: {task} \n test_case: {test_list[0]}\n 코드에 테스트 케이스는 포함하지 마세요."

    result = await create_source_code(prompt_with_name)
    code = getattr(result['private'], "executed_code", None)
    
    num_passed, num_test, num_error = run_tests_exec(code, test_list)
    
    # 각 테스트 케이스의 실행 시간 계산 및 저장
    case_execution_time = time.time() - case_start_time
    execution_times.append(case_execution_time)

    total_num_passed += num_passed
    total_num_test += num_test
    total_num_error += num_error

    if (num_error > 0) or (num_passed < num_test):
        case_err.append([task, code_ground_truth, test_list])

    print(f"Case execution time: {case_execution_time:.2f}s")
    print(f"accuracy : {total_num_passed/total_num_test * 100:.2f}% passed : {total_num_passed}, num test : {total_num_test}, num err : {total_num_error}")
    print(f"Average execution time so far: {statistics.mean(execution_times):.2f}s")

# 전체 실행 통계 출력
total_time = time.time() - start_time
avg_time = statistics.mean(execution_times)
median_time = statistics.median(execution_times)

print("\nBenchmark Summary:")
print(f"Total execution time: {total_time/60:.2f} minutes")
print(f"Average execution time per case: {avg_time:.2f} seconds")
print(f"Median execution time: {median_time:.2f} seconds")
print(f"Fastest case: {min(execution_times):.2f} seconds")
print(f"Slowest case: {max(execution_times):.2f} seconds")
print(f"Final accuracy: {total_num_passed/total_num_test * 100:.2f}%") 

Benchmark started at: 2025-04-25 14:52:46
Case execution time: 12.98s
accuracy : 86.28% passed : 805, num test : 933, num err : 32
Average execution time so far: 15.29s
Case execution time: 9.09s
accuracy : 86.32% passed : 808, num test : 936, num err : 32
Average execution time so far: 15.27s
Case execution time: 10.16s
accuracy : 86.37% passed : 811, num test : 939, num err : 32
Average execution time so far: 15.25s


In [5]:
case_err

[['Write a python function to find the first repeated character in a given string.',
  'def first_repeated_char(str1):\r\n  for index,c in enumerate(str1):\r\n    if str1[:index+1].count(c) > 1:\r\n      return c \r\n  return "None"',
  ['assert first_repeated_char("abcabc") == "a"',
   'assert first_repeated_char("abc") == "None"',
   'assert first_repeated_char("123123") == "1"']],
 ['Write a function to get a lucid number smaller than or equal to n.',
  'def get_ludic(n):\r\n\tludics = []\r\n\tfor i in range(1, n + 1):\r\n\t\tludics.append(i)\r\n\tindex = 1\r\n\twhile(index != len(ludics)):\r\n\t\tfirst_ludic = ludics[index]\r\n\t\tremove_index = index + first_ludic\r\n\t\twhile(remove_index < len(ludics)):\r\n\t\t\tludics.remove(ludics[remove_index])\r\n\t\t\tremove_index = remove_index + first_ludic - 1\r\n\t\tindex += 1\r\n\treturn ludics',
  ['assert get_ludic(10) == [1, 2, 3, 5, 7]',
   'assert get_ludic(25) == [1, 2, 3, 5, 7, 11, 13, 17, 23, 25]',
   'assert get_ludic(45) == [